# Imports

In [1]:
import numpy  as np
import pandas as pd

from functools import reduce

## Loading Datasets

In [2]:
df = pd.read_csv('../../data/raw/CAGEDMOV202302.txt', sep=';', decimal=',')

In [3]:
df.head()

,competênciamov,região,uf,município,seção,subclasse,saldomovimentação,cbo2002ocupação,categoria,graudeinstrução,...,indtrabintermitente,indtrabparcial,salário,tamestabjan,indicadoraprendiz,origemdainformação,competênciadec,indicadordeforadoprazo,unidadesaláriocódigo,valorsaláriofixo
0,202302,4,42,420910,M,6920601,1,413110,101,8,...,0,0,1571.00,4,0,1,202302,0,5,1571.00
1,202302,4,42,421930,G,4744005,-1,411005,101,7,...,0,0,2104.16,3,0,1,202302,0,5,2104.16
2,202302,2,28,280030,H,4921301,-1,782410,101,4,...,0,0,2329.75,9,0,1,202302,0,5,2329.75
3,202302,3,31,310620,E,3811400,1,514215,106,6,...,0,0,1415.13,9,0,1,202302,0,5,1415.13
4,202302,3,35,355030,M,7112000,1,514225,101,7,...,0,0,1603.99,10,0,1,202302,0,5,1603.99


# Aggregation nunique

In [4]:
def mode(arr):

    """
    Returns the most frequent value in a numpy array.
    """
    
    arr = list(arr)
    
    return max(set(arr), key=arr.count)

In [5]:
def agg_unique_categorical(df, columns_to_aggregate, primitive, groupby_cols):
    
    dfs = []
    
    for col in columns_to_aggregate:
        
        df_temp = (
            df.groupby(groupby_cols)
            .agg(aux_name=(col, primitive)).reset_index()
        )
        
        if callable(primitive):
            primitive_name = primitive.__name__
        
        else:
            primitive_name = primitive

        df_temp.columns = groupby_cols + [primitive_name + "_" + col]
        
        dfs.append(df_temp)

    df_agg = reduce(lambda df1, df2: pd.merge(df1, df2, on=groupby_cols, how="outer"), dfs)
    
    return df_agg

In [6]:
df.groupby(['uf'])['seção'].agg(
    secao_nunique=('nunique'),
    secao_mode=(mode),
)

,secao_nunique,secao_mode
uf,,
11,19,G
12,19,G
13,20,G
14,20,G
15,20,G
16,19,G
17,19,G
21,19,G
22,19,G


In [7]:
df_aux = df.groupby(['seção']).agg(
    {
        'uf': ['nunique', mode]
    }
)

In [8]:
df_aux

uf     
      nunique mode
seção             
A          28   35
B          28   31
C          28   35
D          27   35
E          27   35
F          28   35
G          28   35
H          28   35
I          28   35
J          27   35
K          27   35
L          28   35
M          28   35
N          28   35
O          27   35
P          27   35
Q          28   35
R          28   35
S          28   35
T          19   43
U           6   53

In [9]:
df_aux.columns = ['uf_' + name for name in df_aux.columns.droplevel()]

In [10]:
df_aux.reset_index()

,seção,uf_nunique,uf_mode
0,A,28,35
1,B,28,31
2,C,28,35
3,D,27,35
4,E,27,35
5,F,28,35
6,G,28,35
7,H,28,35
8,I,28,35
9,J,27,35


In [11]:
def categorical_mode(arr):

    """
    Returns the most frequent value in a numpy array.
    """
    
    arr = list(arr)
    
    return max(set(arr), key=arr.count)


def agg_cat(df, groupby, variables):

    """
    Function to calculate nunique and mode from a Series

    Parameters
    ----------
    df : DataFrame
        Initial DataFrame

    groupby : List
        List of variables to be groupped.

    variables : List
        List of variables to apply the functions.
    
    Returns
    -------
    df_agg : DataFrame
        Result DataFrame. 
    """

    list_funcs = ['nunique', categorical_mode]

    dict_funcs = {var: list_funcs for var in variables}

    df_agg = df.groupby(groupby).agg(dict_funcs)

    df_agg.columns = [f'{col[0]}_{col[1]}' for col in df_agg.columns]

    return df_agg.reset_index()

In [12]:
agg_cat(df, ['seção'], ['uf'])

,seção,uf_nunique,uf_categorical_mode
0,A,28,35
1,B,28,31
2,C,28,35
3,D,27,35
4,E,27,35
5,F,28,35
6,G,28,35
7,H,28,35
8,I,28,35
9,J,27,35
